<font color=darkblue> 

## Описание задачи
___
    
### Нужно решить, где бурить новую скважину.
#### Шаги для выбора локации:
 - В избранном регионе собирают характеристики для скважин: качество нефти и объём её запасов;
 - Строят модель для предсказания объёма запасов в новых скважинах;
 - Выбирают скважины с самыми высокими оценками значений;
 - Определяют регион с максимальной суммарной прибылью отобранных скважин.

## Описание данных

 - id — уникальный идентификатор скважины;
 - f0, f1, f2 — три признака точек (неважно, что они означают, но сами признаки значимы);
 - product — объём запасов в скважине (тыс. баррелей).

## Вызываю библиотеки

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from numpy.random import RandomState
from scipy import stats as st

<font color=darkblue> 
# Подготовка данных

## 1. Открываю датасеты

In [2]:
region_1 = pd.read_csv('geo_data_0.csv')
region_2 = pd.read_csv('geo_data_1.csv')
region_3 = pd.read_csv('geo_data_2.csv')

## 2. Обучу модель, получу RMSE и среднее

In [3]:
def model(df):
    features = df.drop(['product', 'id'], axis = 1)
    target = df['product']
    features_train, features_valid, target_train, target_valid = train_test_split(
        features, target, test_size=0.25, random_state=12345)
    model = LinearRegression(n_jobs= -1).fit(features_train, target_train)
    predictions = model.predict(features_valid)
    df['predicted'] = model.predict(features)
    mean = predictions.mean()
    print('Среднее предсказанного', predictions.mean())
    print('Среднее выборки', target_train.mean()) 
    print('RMSE', mean_squared_error(target_valid, predictions)**0.5)
    return target, predictions, mean

### Регион 1

In [4]:
target_1, predictions_1, mean_1 = model(region_1)

Среднее предсказанного 92.59256778438038
Среднее выборки 92.6404677530568
RMSE 37.5794217150813


In [5]:
region_1.head()

,id,f0,f1,f2,product,predicted
0,txEyH,0.705745,-0.497823,1.221170,105.280062,95.461973
1,2acmU,1.334711,-0.340164,4.365080,73.037750,116.227394
2,409Wp,1.022732,0.151990,1.419926,85.265647,88.750254
3,iJLyR,-0.032172,0.139033,2.978566,168.620776,95.419237
4,Xdl7t,1.988431,0.155413,4.751769,154.036647,114.138969


### Регион 2

In [6]:
target_2, predictions_2, mean_2 = model(region_2)

Среднее предсказанного 68.72854689544602
Среднее выборки 68.85895465857708
RMSE 0.893099286775616


In [7]:
region_2.head()

,id,f0,f1,f2,product,predicted
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103,3.853530
1,62mP7,14.272088,-3.475083,0.999183,26.953261,26.592376
2,vyE1P,6.263187,-5.948386,5.001160,134.766305,135.665691
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408,138.544872
4,AHL4O,12.702195,-8.147433,5.004363,134.766305,134.867164


### Регион 3

In [8]:
target_3, predictions_3, mean_3 = model(region_3)

Среднее предсказанного 94.96504596800489
Среднее выборки 95.03858906371578
RMSE 40.02970873393434


In [9]:
region_3.head()

,id,f0,f1,f2,product,predicted
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673,75.968506
1,WJtFt,0.262778,0.269839,-2.530187,56.069697,66.329664
2,ovLUW,0.194587,0.289035,-5.586433,62.871910,48.880039
3,q6cA6,2.236060,-0.553760,0.930038,114.572842,86.178035
4,WPMUX,-0.515993,1.716266,5.899011,149.600746,114.363434


<font color=darkgreen> 

# Вывод
 
 ###  Самый маленький разброс у региона №2, но у него и самые низкие средние предсказанные прибыли, возможно у него будут самые низкие риски или выше стабильность.

## 3. Подготовка к рассчету прибыли

### Норма безубыточности для скважины (барелли)

In [10]:
budget = 10**9
chink_numbers = 10**4
profit = 45*10**4
all_chink_vol = (budget/chink_numbers)/profit

In [11]:
all_chink_vol

0.2222222222222222

### Норма безубыточности для 200 скважин,  барель

In [12]:
volume_200 = (budget/200)/profit

In [13]:
volume_200

11.11111111111111

### Валовая средняя прибыль для скважины по локации

In [14]:
def income(x):
    income = round((x*profit) - (budget/profit))
    return income

In [15]:
print('Валовая локации 1:',income(predictions_1.mean()), 'Валовая локации 2:', 
      income(predictions_2.mean()), 'Валовая локации 1:',income(predictions_3.mean()))

Валовая локации 1: 41664433.0 Валовая локации 2: 30925624.0 Валовая локации 1: 42732048.0


<font color=darkgreen> 

# Вывод
 
 ###  Ожидаемо самая низкая средняя валовая прибыль у региона 2, но мы не знаем выбросов по регионам 1 и 3, возможно там самые богатые месторождения. Проверим медианы, чтобы подтвердить теорию.

In [16]:
region_1['product'].median()

91.84997204396296

In [17]:
region_2['product'].median()

57.08562464628662

In [18]:
region_3['product'].median()

94.92561258990246

<font color=darkgreen> 
 
 ###  Медианы близки, а это значит что экстремально больших или малых значений, способных повлиять на среднее, в большом количестве в выборке нет.

## 4. Прибыль по выбранным скважинам

In [19]:
def revenue(df):
    sort_predicted = df.sort_values('predicted', ascending = False).head(200)
    sort_real = df[df.index.isin(sort_predicted.index)]
    revenue = sort_real['product'].sum()*profit
    return revenue

### Регион 1

In [20]:
print ('Доход по предсказаниям', revenue(region_1))


Доход по предсказаниям 13494104192.112015


### Регион 2

In [21]:
print ('Доход по предсказаниям', revenue(region_2))

Доход по предсказаниям 12415086696.68151


### Регион 3

In [22]:
print ('Доход по предсказаниям', revenue(region_3))

Доход по предсказаниям 12571410631.960928


<font color=darkgreen> 

# Вывод
 
 ###  В случае с топовыми месторождениями картина изменилась. Регион 2 теперь не на много уступает остальным, но как мы помним у него меньший разброс. Проверим разброс по топовым значениям.

In [23]:
def RMSE200(df):
    sort_predicted = df.sort_values('predicted', ascending = False).head(200)
    sort_real = df[df.index.isin(sort_predicted.index)]
    rmse = mean_squared_error(sort_real['product'], sort_real['predicted'])**0.5
    return rmse

In [24]:
print('RMSE200 регион 1', RMSE200(region_1))

RMSE200 регион 1 27.63077525318097


In [25]:
print('RMSE регион 2', RMSE200(region_2))

RMSE регион 2 1.243325644367077


In [26]:
print('RMSE регион 3', RMSE200(region_3))

RMSE регион 3 32.540791245595166


<font color=darkgreen> 
 
 ###  В случае с топовыми месторождениями разброс региона 2 увеличился, но тем не менее остается на порядки выше, чем у конкурентов.

## 5. Риски и прибыль для каждого региона

In [27]:
def risks_profit(df):
    alpha = .95
    random_state = np.random.RandomState(12345)
    bootstrap_samples = 1000
    target = df['product']
    predictions = df['predicted']
    profits = []
    riscs = []
    for i in range (bootstrap_samples):
        target_subsample = target.sample(500, replace = True, random_state = random_state)
        data = df[df.index.isin(target_subsample.index)]
        local_rev = revenue(data)
        profits.append(local_rev - budget)
        riscs.append((local_rev - budget) < 0)
    profits = pd.Series(profits)
    riscs = pd.Series(riscs)
    mean_profit = profits.mean()
    risc_percent = riscs.mean()
    confidence_interval = st.t.interval(alpha, len(profits)-1, loc=profits.mean(), scale=profits.sem())
    print('Средняя прибыль', mean_profit)
    print('Доверительный интервал', confidence_interval)
    print('Риск убытков', risc_percent,'%')
    

In [28]:
risks_profit(region_1)

Средняя прибыль 9420090221.952744
Доверительный интервал (9403530978.684, 9436649465.221487)
Риск убытков 0.0 %


In [29]:
risks_profit(region_2)

Средняя прибыль 9441557582.849234
Доверительный интервал (9429306245.729893, 9453808919.968575)
Риск убытков 0.0 %


In [30]:
risks_profit(region_3)

Средняя прибыль 9370724217.804531
Доверительный интервал (9353708565.596891, 9387739870.01217)
Риск убытков 0.0 %


<font color=darkgreen> 

# Вывод
 
 ###  Все регионы перспективны для разработки и дадут близкую прибыль, а риски равны 0. Если затарты на разработку сопоставимы и условия эдентичны, то наиболее привлекательные месторождения расположились в порядке: 
### - Регион 2
### - Регион 1
### - Регион 3
### Помимо прочьего регион 2 будет давать равномерную загрузку по скважинам.